In [1]:
from gensim import corpora, models, similarities
import gensim
import pandas as pd
import pymysql

In [2]:
stopwords=pd.read_csv("./rowdata//stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values

In [3]:
connect = pymysql.connect(host = "localhost", port = 3306, user = "root", password = "123456", database = 'processSentiment', charset='utf8')
sql = 'select * from BiliComment_NoRepeat'
df_senti=pd.read_sql(sql, connect)

In [4]:
df_senti.head()

,keyOrder,uname,mid,ctime,message,text_confidence,text_pos_score,text_neg_score,likehah,rcount,sex,current_level,sign,plat
0,1,Bili_12156495,12156495,2019-08-03 16:38:26,嗯？沙发？。,0.906338,0.042148,0.957852,0,0,男,5,,3
1,2,丿红红火火恍恍惚惚丶丶,42777650,2019-08-03 16:38:26,好早。,0.995621,0.998029,0.001971,0,0,保密,3,,2
2,3,舊夢護城,389411339,2019-08-03 16:38:26,前排。,0.919610,0.963824,0.036176,0,0,男,2,,2
3,4,DYJ0613,8923364,2019-08-03 16:38:27,发现错别字！。,0.263086,0.331611,0.668389,0,0,女,5,这是一个很认真的简介,2
4,5,oldyellow君,205191022,2019-08-03 16:38:27,你的名字归我啦[害羞]。,0.999312,0.999690,0.000309,2,0,保密,5,关注过一个亿女装\n就是永不女装的意思哈哈哈哈哈哈\n,2


In [5]:
df = pd.DataFrame(df_senti['message'])

In [6]:
df.head()

,message
0,嗯？沙发？。
1,好早。
2,前排。
3,发现错别字！。
4,你的名字归我啦[害羞]。


In [7]:
import jieba
import pandas as pd


df = df.dropna()
lines=df.message.values.tolist()

sentences=[]

for line in lines:
    littlesentence = []  ## 课程代码部分有问题，下面作修改
    try:
        segs=jieba.lcut(line)
        segs = filter(lambda x:len(x)>1, segs)# 过滤出 segs 中 长度大于1的字符串
        segs = filter(lambda x:x not in stopwords, segs)# 过滤停用词
        for seg in segs:
            littlesentence.append(seg)
        sentences.append(littlesentence)
    except Exception:
        continue

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/tb/k5xqd901235gk634ymvtk5140000gn/T/jieba.cache
Loading model cost 0.890 seconds.
Prefix dict has been built successfully.


In [8]:
for i in sentences[:5]:
    print(i)

['沙发']
[]
['前排']
['发现', '错别字']
['名字', '害羞']


## 词袋模型

In [9]:
dictionary = corpora.Dictionary(sentences)
corpus = [dictionary.doc2bow(sentence) for sentence in sentences]

## LDA建模


In [10]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)

## 我们查一下第3号分类，其中最常出现的单词是：

In [11]:
print(lda.print_topic(3, topn=10))

0.661*"敬汉卿" + 0.039*"热度" + 0.023*"律师" + 0.022*"证据" + 0.022*"捂脸" + 0.016*"nb" + 0.016*"2018" + 0.014*"撤销" + 0.011*"一句" + 0.007*"提交"


## 我们把所有的主题打印出来看看

In [12]:
for topic in lda.print_topics(num_topics=20, num_words=8):
    print(topic[1])

0.371*"公司" + 0.061*"投币" + 0.054*"恶心" + 0.046*"吃瓜" + 0.044*"字体" + 0.035*"一波" + 0.034*"啊啊啊" + 0.032*"有人"
0.745*"支持" + 0.162*"维权" + 0.010*"版权" + 0.009*"增加" + 0.007*"顶一下" + 0.004*"拥有" + 0.003*"打击" + 0.003*"奇怪"
0.906*"加油" + 0.007*"2019" + 0.007*"举报" + 0.005*"有限公司" + 0.005*"我要" + 0.005*"法院" + 0.004*"团团" + 0.004*"知识产权"
0.661*"敬汉卿" + 0.039*"热度" + 0.023*"律师" + 0.022*"证据" + 0.022*"捂脸" + 0.016*"nb" + 0.016*"2018" + 0.014*"撤销"
0.376*"视频" + 0.176*"粉丝" + 0.042*"流氓" + 0.032*"平台" + 0.022*"公司" + 0.021*"知名" + 0.017*"支付宝" + 0.016*"UP"
0.280*"真的" + 0.280*"出名" + 0.053*"好像" + 0.044*"这家" + 0.030*"Up" + 0.029*"记得" + 0.020*"改名字" + 0.014*"想不到"
0.164*"打气" + 0.091*"生气" + 0.081*"哈哈哈哈" + 0.081*"维护" + 0.075*"jhq" + 0.060*"权益" + 0.050*"共青团中央" + 0.030*"音娘"
0.241*"doge" + 0.154*"事情" + 0.078*"冲冲" + 0.068*"第一次" + 0.058*"2233" + 0.056*"第一" + 0.049*"这是" + 0.045*"永远"
0.284*"播放" + 0.159*"点赞" + 0.081*"汉卿" + 0.050*"兄弟" + 0.040*"一杯" + 0.039*"收藏" + 0.020*"发生" + 0.019*"吃惊"
0.183*"希望" + 0.165*"关注" + 0.085*"你好" + 0.060*"保护" + 0.055